In [ ]:
import joblib
import numpy as np
import pandas as pd
import hashlib
import json
import base64
from time import time
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization, hashes
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.callbacks import EarlyStopping
from web3 import Web3
from solcx import compile_source
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
public_key = private_key.public_key()

symmetric_key = Fernet.generate_key()
cipher_suite = Fernet(symmetric_key)

def send_email_to_admin(subject, body):
    sender_email = "test1@gmail.com"  
    receiver_email = "test2@gmail.com"  
    password = "AAAA"  

    message = MIMEMultipart()
    message["From IDS"] = sender_email
    message["To admin"] = receiver_email
    message["Subject alart"] = subject
    message.attach(MIMEText(body, "plain"))

    try:
        server = smtplib.SMTP("smtp.gmail.com", 465)  
        server.starttls()
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, message.as_string())
        server.quit()
        print("Email sent successfully.")
    except Exception as e:
        print(f"Error sending email: {e}")

def encrypt_data_combined(data):
    data_str = json.dumps(data)
    encrypted_data = cipher_suite.encrypt(data_str.encode())

    encrypted_symmetric_key = public_key.encrypt(
        symmetric_key,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

    encrypted_package = {
        'key': base64.b64encode(encrypted_symmetric_key).decode('utf-8'),
        'data': base64.b64encode(encrypted_data).decode('utf-8')
    }

    return json.dumps(encrypted_package)

def decrypt_data_combined(encrypted_package):
    package = json.loads(encrypted_package)

    encrypted_symmetric_key = base64.b64decode(package['key'])
    decrypted_symmetric_key = private_key.decrypt(
        encrypted_symmetric_key,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

    cipher_suite = Fernet(decrypted_symmetric_key)
    encrypted_data = base64.b64decode(package['data'])
    decrypted_data = cipher_suite.decrypt(encrypted_data).decode('utf-8')

    return json.loads(decrypted_data)

def sign_data(data):
    data_str = json.dumps(data)
    signature = private_key.sign(
        data_str.encode(),
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    return base64.b64encode(signature).decode('utf-8')

def verify_signature(data, signature):
    data_str = json.dumps(data)
    try:
        public_key.verify(
            base64.b64decode(signature.encode('utf-8')),
            data_str.encode(),
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        return True
    except:
        return False

class Blockchain:
    def __init__(self):
        self.chain = []
        self.current_transactions = []
        self.difficulty = 2  
        self.new_block(previous_hash='1', proof=100)

    def new_block(self, proof, previous_hash=None):
        block = {
            'index': len(self.chain) + 1,
            'timestamp': time(),
            'transactions': self.current_transactions,
            'proof': proof,
            'previous_hash': previous_hash or self.hash(self.chain[-1]),
        }
        self.current_transactions = []
        self.chain.append(block)
        return block

    def new_transaction(self, sender, recipient, amount, signature):
        self.current_transactions.append({
            'sender': sender,
            'recipient': recipient,
            'amount': amount,
            'signature': signature
        })
        return self.last_block['index'] + 1

    def proof_of_work(self, last_proof):
        proof = 0
        start_time = time()
        while not self.valid_proof(last_proof, proof):
            proof += 1
            if time() - start_time > 10:  
                self.difficulty += 1  
                start_time = time()
            elif time() - start_time < 5 and self.difficulty > 1:
                self.difficulty -= 1 
                start_time = time()
        return proof

    def valid_proof(self, last_proof, proof):
        guess = f'{last_proof}{proof}'.encode()
        guess_hash = hashlib.sha256(guess).hexdigest()
        return guess_hash[:self.difficulty] == "0" * self.difficulty

    @staticmethod
    def hash(block):
        block_string = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(block_string).hexdigest()

    @property
    def last_block(self):
        return self.chain[-1]

def clean_and_select_features(X, y, max_features=50):
    if isinstance(X, pd.DataFrame):
        X = X.values

    X = np.where(np.isinf(X), np.nan, X)
    X = np.nan_to_num(X, nan=0.0)

    selector = SelectFromModel(RandomForestClassifier(n_estimators=200), max_features=min(max_features, X.shape[1]), threshold=-np.inf)
    selector.fit(X, y)
    X_selected = selector.transform(X)

    print(f"Features selected: {X_selected.shape[1]} out of {X.shape[1]} original features.")
    
    return X_selected, X_selected.shape[1]

data_path_1 = 'data2019.csv'
data_path_2 = 'IoT2023DDoS.csv'

df_test1 = pd.read_csv(data_path_1)
df_test2 = pd.read_csv(data_path_2)

X_test1 = df_test1.drop(columns=['Label'])
y_test1 = df_test1['Label']
X_test1 = X_test1.apply(pd.to_numeric, errors='coerce').fillna(0)
X_test1 = np.where(np.isinf(X_test1), np.nan, X_test1)
X_test1 = np.nan_to_num(X_test1, nan=0.0)
y_test1 = pd.factorize(y_test1)[0]

X_test2 = df_test2.drop(columns=['label'])
y_test2 = df_test2['label']
X_test2 = X_test2.apply(pd.to_numeric, errors='coerce').fillna(0)
X_test2 = np.where(np.isinf(X_test2), np.nan, X_test2)
X_test2 = np.nan_to_num(X_test2, nan=0.0)
y_test2 = pd.factorize(y_test2)[0]

def adjust_features(X, expected_features):
    current_features = X.shape[1]
    if current_features < expected_features:
        additional_features = expected_features - current_features
        X = np.hstack([X, np.zeros((X.shape[0], additional_features))])
    elif current_features > expected_features:
        X = X[:, :expected_features]
    return X

model_path = "partial_fitIoT2023.pkl"
model_data = joblib.load(model_path)

if isinstance(model_data, dict):
    model = model_data.get('model', None)  
    if model is None:
        raise ValueError("المفتاح 'model' غير موجود في قاموس النموذج.")
else:
    model = model_data

expected_features = model.n_features_in_

X_test1 = adjust_features(X_test1, expected_features)
X_test2 = adjust_features(X_test2, expected_features)

def batch_data(data, batch_size):
    for i in range(0, len(data), batch_size):
        yield data[i:i + batch_size]

def detect_and_record_with_encryption_in_batches(model, data, blockchain, batch_size=100):
    batch_index = 0
    for batch in batch_data(data, batch_size):
        predictions = model.predict(batch)
        for index, prediction in enumerate(predictions):
            transaction_data = {
                'index': batch_index * batch_size + index,
                'prediction': int(prediction) if np.isscalar(prediction) else int(prediction[0]),
                'data': batch[index].tolist()  
            }
            encrypted_data = encrypt_data_combined(transaction_data)
            signature = sign_data(transaction_data)
            blockchain.new_transaction(sender="Test Data", recipient="Blockchain", amount=encrypted_data, signature=signature)
        batch_index += 1
        blockchain.new_block(proof=blockchain.proof_of_work(blockchain.last_block['proof']))
        if batch_index % 10 == 0:  
            print(f"Processed batch {batch_index}")

def execute_smart_contract(transaction, ddos_counter, time_frame_start, email_sent):
    current_time = time()

    if transaction['prediction'] == 1:
        ddos_counter += 1

        if ddos_counter > 3 and (current_time - time_frame_start) < 60 and not email_sent:  # على سبيل المثال 60 ثانية
            print("Threat Detected: Multiple DDoS attacks in a short period!")
            send_email_to_admin("Alert: Multiple DDoS Attacks Detected", "Multiple DDoS attacks have been detected in a short period. Immediate action is required.")
            email_sent = True
        elif (current_time - time_frame_start) >= 60:
            ddos_counter = 0
            time_frame_start = current_time
            email_sent = False
    else:
        ddos_counter = 0
        time_frame_start = current_time
        email_sent = False

    return ddos_counter, time_frame_start, email_sent

def audit_and_predictive_analysis(blockchain):
    ddos_counter = 0
    time_frame_start = time()
    email_sent = False
    
    for block in blockchain.chain:
        print(f"Audit Block {block['index']} - Transactions: {len(block['transactions'])}")
        for transaction in block['transactions']:
            decrypted_transaction = decrypt_data_combined(transaction['amount'])
            if verify_signature(decrypted_transaction, transaction['signature']):
                ddos_counter, time_frame_start, email_sent = execute_smart_contract(decrypted_transaction, ddos_counter, time_frame_start, email_sent)
            else:
                print("Invalid signature detected!")

def retrain_model(blockchain, model):
    print("Retraining model using blockchain data...")

    X_train = np.empty((0, model.n_features_in_))
    y_train = np.array([])

    for block in blockchain.chain:
        for transaction in block['transactions']:
            decrypted_data = decrypt_data_combined(transaction['amount'])
            decrypted_features = np.array(decrypted_data['data']).reshape(1, -1)
            
            if decrypted_features.shape[1] < model.n_features_in_:
                additional_features = model.n_features_in_ - decrypted_features.shape[1]
                decrypted_features = np.hstack([decrypted_features, np.zeros((decrypted_features.shape[0], additional_features))])
            elif decrypted_features.shape[1] > model.n_features_in_:
                decrypted_features = decrypted_features[:, :model.n_features_in_]
            
            X_train = np.vstack([X_train, decrypted_features])
            y_train = np.append(y_train, decrypted_data['prediction'])

    X_train, selected_features_count = clean_and_select_features(X_train, y_train)

    if X_train.shape[1] < model.n_features_in_:
        additional_features = model.n_features_in_ - X_train.shape[1]
        X_train = np.hstack([X_train, np.zeros((X_train.shape[0], additional_features))])
    elif X_train.shape[1] > model.n_features_in_:
        X_train = X_train[:, :model.n_features_in_]

    scores = cross_val_score(model, X_train, y_train, cv=5)
    print(f"Cross-Validation Accuracy: {np.mean(scores) * 100:.2f}%")

    model.partial_fit(X_train, y_train)
    print("Model retrained successfully.")
    
    return model, selected_features_count

blockchain = Blockchain()
عدد_الجولات = 1 
accuracy_list_1 = []
accuracy_list_2 = []
features_list = []
confusion_matrices_1 = []
confusion_matrices_2 = []

best_accuracy_test1 = 0
best_accuracy_test2 = 0
best_model = model

for round_num in range(عدد_الجولات):
    detect_and_record_with_encryption_in_batches(model, X_test1, blockchain, batch_size=100)
        detect_and_record_with_encryption_in_batches(model, X_test2, blockchain, batch_size=100)
    model, selected_features_count = retrain_model(blockchain, model)
    
    audit_and_predictive_analysis(blockchain)
    
    y_pred1 = model.predict(X_test1)
    accuracy1 = accuracy_score(y_test1, y_pred1)
    cm1 = confusion_matrix(y_test1, y_pred1)
    
    y_pred2 = model.predict(X_test2)
    accuracy2 = accuracy_score(y_test2, y_pred2)
    cm2 = confusion_matrix(y_test2, y_pred2)
    
    print(f"Accuracy after round {round_num + 1} on test set 1: {accuracy1 * 100:.2f}%")
    print(f"Accuracy after round {round_num + 1} on test set 2: {accuracy2 * 100:.2f}%")
    print(f"Number of features selected after round {round_num + 1}: {selected_features_count}")
    print(f"Confusion Matrix for test set 1 after round {round_num + 1}:\n{cm1}")
    print(f"Confusion Matrix for test set 2 after round {round_num + 1}:\n{cm2}")

    if accuracy1 > best_accuracy_test1 and accuracy2 > best_accuracy_test2:
        best_accuracy_test1 = accuracy1
        best_accuracy_test2 = accuracy2
        best_model = model
        print("Model updated with new best accuracy.")
    else:
        print("Model not updated. Previous model retained.")

    accuracy_list_1.append(accuracy1)
    accuracy_list_2.append(accuracy2)
    features_list.append(selected_features_count)
    confusion_matrices_1.append(cm1)
    confusion_matrices_2.append(cm2)

print("\nFinal Report:")
for i in range(عدد_الجولات):
    print(f"\nRound {i + 1}:")
    print(f"Accuracy on test set 1: {accuracy_list_1[i] * 100:.2f}%")
    print(f"Accuracy on test set 2: {accuracy_list_2[i] * 100:.2f}%")
    print(f"Features selected: {features_list[i]}")
    print(f"Confusion Matrix for test set 1:\n{confusion_matrices_1[i]}")
    print(f"Confusion Matrix for test set 2:\n{confusion_matrices_2[i]}")

def distribute_model_to_devices(model, devices):
    for device in devices:
        print(f"Distributing model to device {device}")
        joblib.dump(best_model, f"{device}_model.joblib")

devices = ["Device_1", "Device_2", "Device_3"]
distribute_model_to_devices(best_model, devices)
